In [ ]:
import datetime as dt
import pandas as pd
import numpy as np

import logging
import pickle
import pytz
import math
import sys
import csv

from pathlib import Path

path_to_here = Path('.').resolve()
path_to_repo = Path('../..').resolve()
path_to_src = path_to_repo / 'src'
path_to_ext = path_to_repo / 'ext'
path_to_data = path_to_repo / 'data' / 'Shared_Data_notinGIT' / 'IEEE13_demo'

sys.path.append(str(path_to_src.resolve()))

export_folder = path_to_here / 'inputs'

from FAST_DERMS_CONTROL.common import FRS_config
from FAST_DERMS_CONTROL.common import input_data_processor as _input_data_processor

In [ ]:
# IEEE 13 OCHRE GONOGO
model_id = '_C07972A7-600D-4AA5-B254-4CAA4263560E'
model_name = 'IEEE13'

__model_data = _input_data_processor(model_id = model_id)

In [ ]:
Sbase = 2e6

# Simulation Times
# Choose whether to use UTC or PST time for simulation
tz = 'US/Pacific'
case_dt0 = dt.datetime(2022, 4, 1, 0, 0, 0)
case_dt0_str = case_dt0.strftime('%Y%m%d')
rt_dt0 = case_dt0 + dt.timedelta(hours=14)

# ISO date override
case_dt0_iso = case_dt0

# Simulation Time
# Simulation time in seconds
# 4 hours
sim_time = 4*60*60 
# Start and End Offsets
simulation_start_offset_minutes = 15
simulation_end_offset_minutes = 15

#Script to load data into PROVEN
load_loadprofile_data = True
reset_loadprofile_data = True

loadprofile_data_filename = 'ieeezipload.player' 
path_to_loadprofile_data = path_to_data / loadprofile_data_filename

load_weather_data = True
reset_weather_data = True

weather_data_filename = 'weather_data.csv'
path_to_weather_data = path_to_data / weather_data_filename

# Load Batteries E0 from Day Ahead
load_battery_E0_data = True

# Ochre Simulation or GridappsD
use_ochre = True
sim_file_gapps = export_folder / 'sim_config_IEEE13.json'
sim_file_ochre = export_folder / 'sim_config_IEEE13_ochre.json'

# Orchestrator
log_orchestrator = logging.DEBUG
mrid_orchestrator = 'FRS'
path_to_orchestrator = path_to_src / 'FAST_DERMS_CONTROL' / 'orchestrator.py'

# FRS PARAMS
## INIT
# Minimum Bus Voltage [pu]
INIT_Vmin = 0.90
# Maximum Bus Voltage [pu]
INIT_Vmax = 1.1 

## DA
# Start timestamp of case
DA_t0 = case_dt0
# Number of timestep in Horizon
DA_n_timestep = 24
# Timestep duration [min]
DA_timestep_period = 60
# Number of scenarios
DA_n_scenario = 20
## Scenario settings
# Number of initial scenario for clustering
DA_n_init = 10000
# Include prices in scenario selection
DA_add_prices = False
# Include Timeseries metrics in scenario selection
DA_TS_metrics = False
# Include Expected value scenario
DA_use_exp_value = True
# Maximum number of iterations for scenario validation
DA_max_loop_nr = 10
# Uncertatinty multiplier
DA_sigma_multiplier = 1
## Model settings
# Thermal approximation
DA_n_thermal = 20
# Remove reserves
DA_remove_reserves = False
# Unity Power Factor Flag
DA_unity_powerfactor = False
# Power cost weight
DA_power_weight = 1
# Transactive Resources weight
DA_transactive_weight = 1
# Reserve weight
DA_reserve_weight = 1
# Loss weight (no losses in this demo)
DA_loss_weight = 0
# Reverse Powerflow weight (None, 0: hard constraint, other number is weight in obj)
DA_reverse_pf_weight = None
# Substation Deviation weight
DA_substation_deviation_weight = 1
# Override for model price (None: use system price)
DA_substation_deviation_price_override = None
# Solver
DA_solver = 'ipopt'

## DET settings
# Unity Power Factor Flag
DET_unity_powerfactor = False
# Power cost weight
DET_power_weight = 1
# Transactive Resources weight
DET_transactive_weight = 1
# Reserve weight
DET_reserve_weight = 1
# Loss weight (no losses in this demo)
DET_loss_weight = 0
# Reverse Powerflow weight (None, 0: hard constraint, other number is weight in obj)
DET_reverse_pf_weight = None
# Substation Deviation weight
DET_substation_deviation_weight = 1
# Override for model price (None: use system price)
DET_substation_deviation_price_override = None
# Reserves Deviation weight
DET_reserves_deviation_weight = 1
# Override for model price (None: use system price)
DET_reserves_deviation_price_override = None
# Solver
DET_solver = 'ipopt'

#### SIMULATION
# Hotstart file (results from DA)
path_to_hotstart_file = path_to_here / 'archive' / f"DayAhead_Process_{case_dt0.strftime('%Y%m%d')}.pkl"

## MPC 
# Start timestamp of the MPC
MPC_t0 = rt_dt0
# Iteration Offset [s.] (to start the MPC computation ahead of time)
MPC_iteration_offset = -9*60
# Number of incremented timestep at each iteration
MPC_n_skip = 1
# Number of timestep in Horizon
MPC_n_timestep = 16
# Timestep duration [min]
MPC_timestep_period = 15
## MPC 
# Additional reserve requirement settings
MPC_opt_R_dist = {}
MPC_opt_R_dist['beta_load'] = 0.9
MPC_opt_R_dist['beta_DER'] = 0.9
MPC_opt_R_dist['R_sigma_up'] = 1.5
MPC_opt_R_dist['R_sigma_dn'] = 1.5
# Uncertatinty multiplier
MPC_sigma_multiplier = 1
## Model Settings
# Thermal approximation
MPC_n_thermal = 20
# Unity Power Factor Flag
MPC_unity_powerfactor = False
# Power cost weight
MPC_power_weight = 1
# Transactive Resources weight
MPC_transactive_weight = 1
# Reserve weight
MPC_reserve_weight = 1
# Weight multiplier for this component of the objective
MPC_substation_deviation_weight = 1
# Override for model price (None: use system price)
MPC_substation_deviation_price_override = None
# Weight multiplier for this component of the objective
MPC_reserves_deviation_weight = 1
# Override for model price (None: use system price)
MPC_reserves_deviation_price_override = None
# PV Curtailment weight
MPC_pv_curtailment_weight = 1
# Battery charge/discharge weight
MPC_battery_charge_discharge_weight = 1
# Battery deviation weight
MPC_battery_deviation_weight = 1000
# flag indicating whether to ensure final battery energy levels are same as determinisitic day ahead problem
MPC_E_BAT_final_flag = False
# flag indicating whether to ensure final EV energy levels are same as determinisitic day ahead problem
MPC_E_EV_final_flag = False
# flag indicating whether to ensure final flexible load energy levels are same as determinisitic day ahead problem
MPC_E_FL_final_flag = False
# Solver
MPC_solver = 'ipopt'

# Network Data
file_all_data = export_folder / f'{model_name}_{case_dt0_str}_data.pkl'
#------------------------- Master Spreadsheet
ms_file= export_folder / 'MS_IEEE13_demo.xlsx'

__model_data.set_OCHRE_MS(ms_file)

# RT controller
use_rt_controller = True
mrid_rt_controller = 'RT_CONTROLLER'
log_rt_controller = logging.DEBUG
path_to_rt_controller = path_to_src / 'FAST_DERMS_CONTROL' / 'Control' / 'DER_RT_Control.py'
period_rt_controller= 60
Kp_rt_controller = 0.75
Ki_rt_controller = 0.0001
Kd_rt_controller = 0.1
rollover_error_rt_controller = True
iso_ramp_rt_controller = False
iso_ramp_n_step_rt_controller = 4
substation_ID = 'node_650'
substation_line_meas = 'PowerTransformer_source_650_Power'
substation_measurement_IDs_rt_controller = __model_data.get_substation_measurement_IDs(substation_ID, substation_line_meas)
PV_measurement_IDs_rt_controller = __model_data.get_PV_measurement_IDs()

# NOT FRS, but part of DEMO
# ISO
mrid_iso = 'ISO'
log_iso = logging.DEBUG
path_to_iso = path_to_src / 'ISO' / 'ISO_main.py'
# in [.s]
dispatch_period_iso = 5*60
use_static_data_iso = True

# TMM
use_tmm = True
mrid_tmm = 'TMM_1'
path_to_bids_tmm = path_to_data / 'TMM_bids.pkl'
log_tmm = logging.DEBUG
path_to_tmm = path_to_ext / 'thems_app' / 'runapp_main.py'
timestep_tmm = 15*60 #s.
iteration_offset_tmm = 10 #s.
num_houses_tmm = 40
models_tmm = {}
for i, t_init in zip(range(1, num_houses_tmm+1), np.arange(20,25,(25-20)/num_houses_tmm)):
    models_tmm[f'Envelope_n{i}'] = {'T Indoor': t_init}
    models_tmm[f'HVAC Cooling_n{i}'] = {'T Setpoint': 21, 'T Upper Limit': 25, 'T Lower Limit': 17}

# Dispatcher
use_dispatcher = True
mrid_dispatcher = 'DISPATCHER'
log_dispatcher = logging.DEBUG
path_to_dispatcher = path_to_ext / 'dispatcher' / 'dispatcher.py'

PV_list_dispatcher = __model_data.get_dispatchable_PV_list()
BATT_list_dispatcher = {}

# Battery Aggregator
use_batt_aggregator = True
mrid_batt_aggregator = 'AGGREGATOR'
log_batt_aggregator = logging.DEBUG
path_to_batt_aggregator = path_to_ext / 'battery_aggregator_app' / 'runapp_main.py' 

# Battery Data
# Aggregates
aggregate_models_batt_aggregator = {
        'BAT2':{'n2b': {'soc': 0.5, 'E': 125, 'P': 50, 'nuC': 0.95, 'nuD': 0.95},
            'n3b': {'soc': 0.5, 'E': 100, 'P': 40, 'nuC': 0.95, 'nuD': 0.95},
            'n4b': {'soc': 0.5, 'E': 150, 'P': 60, 'nuC': 0.95, 'nuD': 0.95}},
    'BAT4':{'n5b': {'soc': 0.5, 'E': 125, 'P': 50, 'nuC': 0.95, 'nuD': 0.95},
            'n6b': {'soc': 0.5, 'E': 125, 'P': 50, 'nuC': 0.95, 'nuD': 0.95},
            'n16b': {'soc': 0.5, 'E': 225, 'P': 90, 'nuC': 0.95, 'nuD': 0.95},
            'n25b': {'soc': 0.5, 'E': 125, 'P': 50, 'nuC': 0.95, 'nuD': 0.95}},
    'BAT5':{'n9b': {'soc': 0.5, 'E': 75, 'P': 30, 'nuC': 0.95, 'nuD': 0.95},
            'n11b': {'soc': 0.5, 'E': 50, 'P': 20, 'nuC': 0.95, 'nuD': 0.95},
            'n31b': {'soc': 0.5, 'E': 50, 'P': 20, 'nuC': 0.95, 'nuD': 0.95},
            'n13b': {'soc': 0.5, 'E': 50, 'P': 20, 'nuC': 0.95, 'nuD': 0.95},
            'n14b': {'soc': 0.5, 'E': 50, 'P': 20, 'nuC': 0.95, 'nuD': 0.95},
            'n15b': {'soc': 0.5, 'E': 50, 'P': 20, 'nuC': 0.95, 'nuD': 0.95},
            'n23b': {'soc': 0.5, 'E': 50, 'P': 20, 'nuC': 0.95, 'nuD': 0.95},
            'n37b': {'soc': 0.5, 'E': 50, 'P': 20, 'nuC': 0.95, 'nuD': 0.95}}}

aggregate_models_batt_aggregator = __model_data.format_OCHRE_BATT_data(aggregate_models_batt_aggregator)

# Utility scale batteries
utility_models_batt_aggregator={
    'BAT1':{'n28b': {'soc': 0.5, 'E': 583.3333333, 'P': 233.33333333, 'nuC': 0.95, 'nuD': 0.95},
            'n29b': {'soc': 0.5, 'E': 583.3333333, 'P': 233.33333333, 'nuC': 0.95, 'nuD': 0.95},
            'n30b': {'soc': 0.5, 'E': 583.3333333, 'P': 233.33333333, 'nuC': 0.95, 'nuD': 0.95}},
    'BAT3':{'n22b': {'soc': 0.5, 'E': 175, 'P': 70, 'nuC': 0.95, 'nuD': 0.95},
            'n38b': {'soc': 0.5, 'E': 175, 'P': 70, 'nuC': 0.95, 'nuD': 0.95}},
}

utility_models_batt_aggregator = __model_data.format_OCHRE_BATT_data(utility_models_batt_aggregator)

# Measurement IDs
meas_ids_batt_aggregator = __model_data.get_BATT_measurement_IDs(aggregate_models_batt_aggregator)
meas_ids_batt_aggregator.update(__model_data.get_BATT_measurement_IDs(utility_models_batt_aggregator))

# MPC Dispatch Controller
use_mpc_dispatch_controller = False
mrid_mpc_dispatch_controller = 'MPC_DISPATCH_CONTROLLER'
log_mpc_dispatch_controller = logging.DEBUG
path_to_mpc_dispatch_controller = path_to_src / 'FAST_DERMS_CONTROL' / 'Control' / 'MPC_Dispatch_Control.py'
period_mpc_dispatch_controller = 0
iteration_offset_mpc_dispatch_controller = 0
DER_ids_mpc_dispatch_controller = ['MICRO_1']
advanced_mpc_dispatch_controller = False

# Microgrid Interface
use_microgrid_interface = False
mrid_microgrid_interface = 'MICROGRID_INTERFACE'
log_microgrid_interface = logging.DEBUG
path_to_microgrid_interface = path_to_ext / 'microgrid_interface' / 'main_app.py'
period_microgrid_interface = 60
iteration_offset_microgrid_interface = 0
microgrid_folder_microgrid_interface = path_to_here / 'microgrid_comms'
MicroGrid_ids_microgrid_interface = {'MICRO_1': "Add_External_ID_if_Needed"}

# ADMS Publisher
use_adms_publisher = True
mrid_adms_publisher = 'ADMS'
log_adms_publisher = logging.DEBUG
path_to_adms_publisher = path_to_ext / 'adms_publisher' / 'main.py'
period_adms_publisher = 60
iteration_offset_adms_publisher = 0
search_folder_adms_publisher = path_to_here / 'adms_input'


In [ ]:
my_FRS_config = FRS_config(sys.modules[__name__])
path_to_config_file = path_to_here / 'FRS_config.json'
my_FRS_config.export_config(path_to_config_file)

# DATA PREPARATION
## Static Data

In [ ]:
static_data = {}

static_data['data_type'] = 'static'
static_data['model_name'] = model_name
static_data['model_ID'] = model_id
static_data['model_file'] = 'IEEE13_demo.xml'
static_data['substation_ID'] = 'node_650'

static_data['Switches'] = {}
static_data['Switches']['switch35'] = {'status' : True}
static_data['Switches']['switch46'] = {'status' : True}

static_data['DERs'] = {}

# BAT1 Utility-scale storage unit (single location)
mRID = 'BAT1'
type = 'BAT'
bus_phase = {'node_692':[0,1,2]}
#kVA 
smax = 700 * 1000
eff_c = 0.95
eff_d = 0.95
#kWh
emax = 1750 * 1000 # 80% is 1400 kWh
gamma = {'node_692':[1.0/3.0, 1.0/3.0, 1.0/3.0]}
#Unused for single inverter
qmin = None 
qmax = None
E_0 = 0.5*emax #SOC * Emax p.u.(manually putting in Sbase since this will have to be replaced to get it off of platform)

static_data['DERs'][mRID] = {'type':type, 'bus_phase':bus_phase, 'smax':smax, 'eff_c':eff_c, 'eff_d':eff_d, 'emax':emax, 'gamma':gamma, 'qmin':qmin, 'qmax':qmax, 'E_0':E_0}

# BAT2 Aggregation of Batteries 
mRID = 'BAT2'
type = 'BAT'
bus_phase = {'node_634':[0,1,2]} #This is an aggregation of units that are at a single buse.  This may read as a three phase single unit though, so there may need to be something added to the model building section that accounts for this, maybe if the "max_capacity" field is active, then you could use that to differentiate.
smax = None #Not really useful for aggregations, though there should be a Pmax.  It is possible that Pmax could just be computed from max capacity across the aggregation.  
qmin = -100 * 1000 #kVAR
qmax = 100 * 1000 #kVAR
eff_c = 0.95
eff_d = 0.95
emax = 375 * 1000 # 80% is 300 kWh
gamma = {'node_634':[50.0, 40.0, 60.0]} #This is actually the max capacity (in KVA) on each phase
E_0 = 0.5*emax

static_data['DERs'][mRID] = {'type':type, 'bus_phase':bus_phase, 'smax':smax, 'eff_c':eff_c, 'eff_d':eff_d, 'emax':emax, 'gamma':gamma, 'qmin':qmin, 'qmax':qmax, 'E_0':E_0}

# BAT3 Utility-scale storage unit (single location)
mRID = 'BAT3'
type = 'BAT'
smax = 140 * 1000 #kVA 
qmin = None #Unused for single inverter
qmax = None #Unused for single inverter
eff_c = 0.95
eff_d = 0.95
emax = 350 * 1000 # 80% is 280 kWh
bus_phase = {'node_646':[0,1]}
gamma = {'node_646':[1.0/2.0, 1.0/2.0, 0.0]}
E_0 = 0.5*emax

static_data['DERs'][mRID] = {'type':type, 'bus_phase':bus_phase, 'smax':smax, 'eff_c':eff_c, 'eff_d':eff_d, 'emax':emax, 'gamma':gamma, 'qmin':qmin, 'qmax':qmax, 'E_0':E_0}

# PV-1  Utility PV
mRID = 'PV1'
type = 'PV'
bus_phase = {'node_646':[1,2]}
pmin = 0
smax = 225 * 1000 #kVA
qmin = None #Unused for single inverter
qmax = None #Unused for single inverter
gamma = {'node_646':[0, 1.0/2.0, 1.0/2.0]}

static_data['DERs'][mRID] = {'type':type, 'bus_phase':bus_phase, 'pmin':pmin, 'smax':smax, 'qmin':qmin, 'qmax':qmax, 'gamma':gamma}

# PV-2  Utility PV
mRID = 'PV2'
type = 'PV'
bus_phase = {'node_611':[2]}
pmin = 0
smax = 250 * 1000 #kVA
qmin = None #Unused for single inverter
qmax = None #Unused for single inverter
gamma = {'node_611':[0, 0, 1.0]}

static_data['DERs'][mRID] = {'type':type,'bus_phase':bus_phase, 'pmin':pmin, 'smax':smax, 'qmin':qmin, 'qmax':qmax, 'gamma':gamma}

# PV-3  Utility PV
mRID = 'PV3'
type = 'PV'
bus_phase = {'node_671':[0,1,2]}
pmin = 0
smax = 375 * 1000 #kVA
qmin = None #Unused for single inverter
qmax = None #Unused for single inverter
gamma = {'node_671':[1.0/3.0, 1.0/3.0, 1.0/3.0]}

static_data['DERs'][mRID] = {'type':type, 'bus_phase':bus_phase, 'pmin':pmin, 'smax':smax, 'qmin':qmin, 'qmax':qmax, 'gamma':gamma}

# BAT4
mRID = 'BAT4'
type = 'BAT'
bus_phase = {'node_611':[2],'node_645':[1,2], 'node_675':[0]}
smax = None #Not really useful for aggregations, though there should be a Pmax.  It is possible that Pmax could just be computed from max capacity across the aggregation.  
qmin = 0 * 1000 #kVAR
qmax = 0 * 1000 #kVAR
eff_c = 0.95
eff_d = 0.95
emax = 600 * 1000 # 80% is 480 kWh
gamma = {'node_611':[0,0,float(90)],'node_645':[0,float(50),float(50)], 'node_675':[float(50),0,0]} #again, based on max capacities.
E_0 = 0.5*emax

static_data['DERs'][mRID] = {'type':type, 'bus_phase':bus_phase, 'smax':smax, 'eff_c':eff_c, 'eff_d':eff_d, 'emax':emax, 'gamma':gamma, 'qmin':qmin, 'qmax':qmax, 'E_0':E_0}

mRID = 'BAT5'
type = 'BAT'
bus_phase = {'node_646': [1,2], 'node_692': [0,1,2], 'node_675': [1], 'node_652': [0], 'node_611': [2]}
smax = None #Not really useful for aggregations, though there should be a Pmax.  It is possible that Pmax could just be computed from max capacity across the aggregation.  
qmin = -100 * 1000 #kVAR
qmax = 100 * 1000 #kVAR
eff_c = 0.95
eff_d = 0.95
emax = 425 * 1000 # 80% is 340 kWh
gamma = {'node_646': [0, 20.0, 20.0], 'node_692': [ 20.0, 20.0, 20.0], 'node_675': [0, 20.0, 0], 'node_652': [30.0, 0, 0], 'node_611':[0,0,20.0]}
E_0 = 0.5*emax

static_data['DERs'][mRID] = {'type':type,'bus_phase':bus_phase, 'smax':smax, 'eff_c':eff_c, 'eff_d':eff_d, 'emax':emax, 'gamma':gamma, 'qmin':qmin, 'qmax':qmax, 'E_0':E_0}

# TMM 
mRID = 'TMM_1'
type = 'TR'
bus_phase = {'node_692':[0,1,2], 'node_634':[0,1,2], 'node_646':[0,1,2], 'node_611':[2],'node_645':[1,2], 'node_675':[0,1], 'node_692': [0,1,2], 'node_652': [0], 'node_611': [2]}
smax = None #Not really useful for aggregations, though there should be a Pmax.  It is possible that Pmax could just be computed from max capacity across the aggregation.  
qmin = -100 * 1000 #kVAR
qmax = 100 * 1000 #kVAR
pmin = 0
pmax = 100 * 1000
price_min = 0
price_max = 100

gamma = {'node_692':[1,1,1], 'node_634':[1,1,1], 'node_646':[1,1,1], 'node_611':[0,0,1],'node_645':[0,1,1], 'node_675':[1,1,0], 'node_692': [1,1,1], 'node_652': [1,0,0], 'node_611': [0,0,1]}
static_data['DERs'][mRID] = {'type':type,'bus_phase':bus_phase, 'smax':smax, 'qmin':qmin, 'qmax':qmax, 'pmin':pmin, 'pmax':pmax, 'price_min':price_min, 'price_max':price_max, 'gamma':gamma}

# Example Loads
static_data['Loads'] = {}
# Load on bus 632
mRID = 'L632'
bus = 'node_632'
static_data['Loads'][mRID] = {'bus':bus}

# Load on bus 645 
mRID = 'L645'
bus = 'node_645'
static_data['Loads'][mRID] = {'bus':bus}

#Load on bus 646
mRID = 'L646'
bus = 'node_646'
static_data['Loads'][mRID] = {'bus':bus}

#Load on bus 634
mRID = 'L634'
bus = 'node_634'
static_data['Loads'][mRID] = {'bus':bus}

#Load on bus 671
mRID = 'L671'
bus = 'node_671'
static_data['Loads'][mRID] = {'bus':bus}

#Load on bus 611
mRID = 'L611'
bus = 'node_611'
static_data['Loads'][mRID] = {'bus':bus}

#Load on bus 652
mRID = 'L652'
bus = 'node_652'
static_data['Loads'][mRID] = {'bus':bus}

#Load on bus 692
mRID = 'L692'
bus = 'node_692'
static_data['Loads'][mRID] = {'bus':bus}

#Load on bus 675
mRID = 'L675'
bus = 'node_675'
static_data['Loads'][mRID] = {'bus':bus}

In [ ]:
export_folder = Path(export_folder)
with open(file_all_data, 'wb') as f:
    pickle.dump(static_data, f)

## Forecasts

In [ ]:
tz_str = 'US/Pacific'
tz = pytz.timezone(tz_str)

fcast_data = {}
fcast_data['data_type'] = 'fcast'
fcast_data['t0'] = tz.localize(case_dt0)
fcast_data['timestep_period'] = 60
fcast_data['n_timestep'] = 24

fcast_data['Switches'] = {}

fcast_data['Switches']['switch35'] = {'status' : True}
fcast_data['Switches']['switch46'] = {'status' : True}

fcast_data['Loads'] = {}

path_to_forecasts = path_to_data / 'forecasts'

#Identify file paths
#path to the node map
Node_map_fn = path_to_forecasts / 'load' / 'node_sequence.csv' 
#path to the forecasted real power mean
Load_mu_fn = path_to_forecasts / 'load' / 'mu_39node.csv' 
#path to the forecasted real power standard deviation
Load_sigma_fn = path_to_forecasts / 'load' / 'sigma_39node.csv' 

# Obtaining the node map:
with open(Node_map_fn, 'r') as f:
    reader = csv.reader(f)
    raw_map = []
    for row in reader:
        raw_map.append(row)

#reconfigure the node map into a list of tuples:
node_map = []
for row in raw_map:
    row_data = row[0].split('-')
    node_map.append(tuple(['L'+row_data[0],int(row_data[1])-1]))

# Open the real power file
with open(Load_mu_fn, 'r') as f:
    reader = csv.reader(f)
    raw_Pmu = []
    for row in reader:
        raw_Pmu.append(row)

# open sigma file
with open(Load_sigma_fn, 'r') as f:
    reader = csv.reader(f)
    raw = []
    for row in reader:
        raw.append(row)

#Manually put in the PF data from the original IEEE13 substation: [L632, L645, L646, L634, L671, L611, L652, L692, L675]
for mRID in static_data['Loads'].keys():
    # power factor, lagging
    #1 is used here when there is no load on the phase.
    if mRID == 'L632':
        example_PF = [0.862, 0.867, 0.865]

    elif mRID == 'L634':
        example_PF = [0.824,0.8,0.8]

    elif  mRID == 'L645':
        example_PF = [1, 0.806, 1]

    elif  mRID == 'L646':
        example_PF = [1, 0.867, 1] 

    elif  mRID == 'L652':
        example_PF = [0.83, 1, 1]

    elif  mRID == 'L671':
        example_PF = [0.868, 0.868, 0.868] 

    elif  mRID == 'L675':
        example_PF = [0.931, 0.75, 0.807] 

    elif  mRID == 'L692':
        example_PF = [1, 1, 0.748] 

    elif  mRID == 'L611':
        example_PF = [1,1, 0.905] 
    else:
        example_PF = [1, 1, 1]

    #find the indices of mRID in node_map:
    indices = [row[0]==mRID for row in node_map]

    #create x_bar
    of_interest = []
    for ind in range(len(raw_Pmu)):
        if indices[ind]:
            of_interest.append([float(P)*1000 for P in raw_Pmu[ind]])
    x_bar = list(np.array(of_interest).transpose())

    #create sigma
    of_interest = []
    for ind in range(len(raw)):
        if indices[ind]:
            of_interest.append([float(P)*1000 for P in raw[ind]])
    sigma = list(np.array(of_interest).transpose())

    fcast_data['Loads'][mRID] = {}
    fcast_data['Loads'][mRID]['PL'] = {'xbar':x_bar, 'sigma':sigma}

    x_bar_q = [[x_bar[t][idx] * math.tan(math.acos(example_PF[idx])) for idx in [0,1,2]]for t in range(len(x_bar))]
    sigma_q = [[sigma[t][idx] * math.tan(math.acos(example_PF[idx])) for idx in [0,1,2]]for t in range(len(sigma))]
    fcast_data['Loads'][mRID]['QL'] = {'xbar':x_bar_q, 'sigma':sigma_q}


#Identify file paths
#path to the forecasted real power mean
PV_mu_fn = path_to_forecasts / 'pv' / 'mu_of_30_PVs.csv'
#path to the forecasted real power standard deviation
PV_sigma_fn = path_to_forecasts / 'pv' / 'sigma_of_30_PVs.csv' 

# Open the real power file
with open(PV_mu_fn) as csvfile:
    raw_mu = list(csv.reader(csvfile))
GeneralPVfcast_mu = np.array([float(x) for x in raw_mu[0]])
#Get the max for normalization in the PV file (We shouldn't be doing this for final forecaster), as a [proxy of the maximum PV power of the forecasted resource]
Pmax4norm = GeneralPVfcast_mu.max()

# open sigma file
with open(PV_sigma_fn) as csvfile:
    raw = list(csv.reader(csvfile))
#create sigma
sigma = [float(x) / Pmax4norm for x in raw[0]]

def get_fcast_PV(Pmax):
    #create x_bar (Pmax in p.u.)
    x_bar = list(GeneralPVfcast_mu * Pmax / Pmax4norm )

    #create sigma
    sigma = [float(x) / Pmax4norm * Pmax for x in raw[0]]
    return x_bar, sigma

fcast_data['DERs'] = {}
for mRID in static_data['DERs'].keys():
    fcast = {}
        #Assumes that Sbase is in kW, returns p.u.
    if mRID == 'BAT1':
        key = 'Emax' #Not a random variable right now.
        x_bar = [0.9*1750*1000]
        sigma = [0]
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}
        key = 'Emin'
        x_bar = [0.1*1750*1000]
        sigma = [0]
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}
        key = 'Pmax'
        x_bar = [700*1000]
        sigma = [0]
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}
        key = 'Pmin'
        x_bar = [-700*1000]
        sigma = [0]
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}

    elif mRID == 'BAT2': #not a random variable
        key = 'Emax'
        x_bar = [0.9 * 375 *1000]
        sigma = [0]
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}
        key = 'Emin'
        x_bar = [0.1 * 375 *1000]
        sigma = [0]
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}
        key = 'Pmax'
        x_bar = [150*1000]
        sigma = [0]
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}
        key = 'Pmin'
        x_bar = [-150*1000]
        sigma = [0]
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}

    elif mRID == 'BAT3': #not a random variable
        key = 'Emax'
        x_bar = [0.9*350*1000]
        sigma = [0]
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}
        key = 'Emin'
        x_bar = [0.1*350*1000]
        sigma = [0]
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}
        key = 'Pmax'
        x_bar = [140*1000]
        sigma = [0]
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}
        key = 'Pmin'
        x_bar = [-140*1000]
        sigma = [0]
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}

    elif mRID == 'BAT4': #not a random variable
        key = 'Emax'
        x_bar = [0.9 * 600 *1000]
        sigma = [0]
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}
        key = 'Emin'
        x_bar = [0.1 * 600 *1000]
        sigma = [0]
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}
        key = 'Pmax'
        x_bar = [240*1000]
        sigma = [0]
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}
        key = 'Pmin'
        x_bar = [-240*1000]
        sigma = [0]
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}

    elif mRID == 'BAT5': #not a random variable
        key = 'Emax'
        x_bar = [0.9 * 425 *1000]
        sigma = [0]
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}
        key = 'Emin'
        x_bar = [0.1 * 425 *1000]
        sigma = [0]
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}
        key = 'Pmax'
        x_bar = [170*1000]
        sigma = [0]
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}
        key = 'Pmin'
        x_bar = [-170*1000]
        sigma = [0]
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}

    elif mRID == 'PV1':
        key = 'Pmax'
        [x_bar, sigma] = get_fcast_PV(Pmax = 225 * 1000)
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}
        key = 'Pmin'
        x_bar = [0]
        sigma = [0]
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}
        key = 'Qmin'
        x_bar = [-0.3*225*1000]
        sigma = [0]
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}
        key = 'Qmax'
        x_bar = [0.3*225*1000]
        sigma = [0]
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}

    elif mRID == 'PV2':
        key = 'Pmax'
        [x_bar, sigma] = get_fcast_PV(Pmax =250 * 1000)
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}
        key = 'Pmin'
        x_bar = [0]
        sigma = [0]
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}
        key = 'Qmin'
        x_bar = [-0.3*250*1000]
        sigma = [0]
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}
        key = 'Qmax'
        x_bar = [0.3*250*1000]
        sigma = [0]
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}

    elif mRID == 'PV3':
        key = 'Pmax'
        [x_bar, sigma] = get_fcast_PV(Pmax = 375 * 1000)
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}
        key = 'Pmin'
        x_bar = [0]
        sigma = [0]
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}
        key = 'Qmin'
        x_bar = [-0.3*375*1000]
        sigma = [0]
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}
        key = 'Qmax'
        x_bar = [0.3*375*1000]
        sigma = [0]
        fcast[key] = {'xbar':x_bar, 'sigma':sigma}

    fcast_data['DERs'][mRID] = fcast


#real prices from ERCOT, AEN Ld Zone, July 17th 2017
fcast = {}
key = 'pi'
x_bar = [20.68, 20.08, 19.28, 18.92, 18.92, 19.64, 19.98, 20.31, 21.61, 24.18, 28.50, 33.01, 37.08, 42.55, 49.65, 52.44, 48.90, 34.91, 31.41, 29.21, 29.85, 25.14, 24.88, 22.70]
sigma = list(np.array(x_bar)*0.25)
fcast[key] = {'xbar':x_bar, 'sigma':sigma}
key = 'pi_rup'
x_bar = [0.95, 0.95, 0.95, 0.95, 0.95, 0.95, 1.50, 1.50, 2.61, 4.60, 7.53, 9.86, 12.91, 15.97, 22.5, 24.62, 20.27, 5.48, 5.17, 5.4, 5.66, 2.61, 2.21, 1.01]
sigma = list(np.array(x_bar)*0.2)
fcast[key] = {'xbar':x_bar, 'sigma':sigma}
key = 'pi_rdn'
x_bar = [3.46, 2.99, 2.99, 2.61, 2.00, 2.11, 2.56, 2.11, 3.46, 3.11, 2.61, 2.21, 1.61, 1.81, 2.61, 3.50, 3.85, 3.02, 2.31, 2.31, 2.61, 5.00, 2.99, 2.99]
sigma = list(np.array(x_bar)*0.2)
fcast[key] = {'xbar':x_bar, 'sigma':sigma}

fcast_data['prices'] = fcast

# Export Configs

## PV + BATT + ADMS + TMM

In [ ]:
case = 'PV_BATT_ADMS_POWER_TMM'
# add an Event to the list of events to be published
# time
adms_constraint_time = rt_dt0 + dt.timedelta(minutes=15)
# File
adms_constraint_file = str(path_to_here / 'adms_input' / 'examples' / 'NMS power constraints.csv')

my_FRS_config._set_param('adms_constraint_time', adms_constraint_time)
my_FRS_config._set_param('adms_constraint_file', adms_constraint_file)
my_FRS_config.use_tmm = True
my_FRS_config.use_adms_publisher = True
my_FRS_config.path_to_hotstart_file = str(path_to_here / 'archive' / f"DayAhead_Process_{case}.pkl")
my_FRS_config.file_all_data = str(export_folder / f'{model_name}_{case}_data.pkl')

path_to_config_file = export_folder / f'FRS_config_{case}.json'
my_FRS_config.export_config(path_to_config_file)
with open(my_FRS_config.file_all_data, 'wb') as f:
    pickle.dump(static_data, f)
with open(my_FRS_config.file_all_data, 'ab') as f:
    pickle.dump(fcast_data, f)

## PV + BATT + TMM

In [ ]:
case = 'PV_BATT_TMM'
my_FRS_config.use_tmm = True
my_FRS_config.use_adms_publisher = False
my_FRS_config.path_to_hotstart_file = str(path_to_here / 'archive' / f"DayAhead_Process_{case}.pkl")
my_FRS_config.file_all_data = str(export_folder / f'{model_name}_{case}_data.pkl')

path_to_config_file = export_folder / f'FRS_config_{case}.json'
my_FRS_config.export_config(path_to_config_file)
with open(my_FRS_config.file_all_data, 'wb') as f:
    pickle.dump(static_data, f)
with open(my_FRS_config.file_all_data, 'ab') as f:
    pickle.dump(fcast_data, f)

## PV + BATT + ADMS (Power)

In [ ]:
case = 'PV_BATT_ADMS_POWER'

# add an Event to the list of events to be published
# time
adms_constraint_time = rt_dt0 + dt.timedelta(minutes=15)
# File
adms_constraint_file = path_to_here / 'adms_input' / 'examples' / 'NMS power constraints.csv'

my_FRS_config._set_param('adms_constraint_time', adms_constraint_time)
my_FRS_config._set_param('adms_constraint_file', adms_constraint_file)
my_FRS_config.use_tmm = False
my_FRS_config.use_adms_publisher = True
my_FRS_config.path_to_hotstart_file = str(path_to_here / 'archive' / f"DayAhead_Process_{case}.pkl")
my_FRS_config.file_all_data = str(export_folder / f'{model_name}_{case}_data.pkl')

#Remove TMM from DER list
try:
    static_data['DERs'].pop(mrid_tmm)
except:
    pass

path_to_config_file = export_folder / f'FRS_config_{case}.json'
my_FRS_config.export_config(path_to_config_file)
with open(my_FRS_config.file_all_data, 'wb') as f:
    pickle.dump(static_data, f)
with open(my_FRS_config.file_all_data, 'ab') as f:
    pickle.dump(fcast_data, f)

## PV + BATT + ADMS (Voltage)

In [ ]:
case = 'PV_BATT_ADMS_VOLTAGE'

# add an Event to the list of events to be published
# time
adms_constraint_time = rt_dt0 + dt.timedelta(minutes=15)
# File
adms_constraint_file = path_to_here / 'adms_input' / 'examples' / 'NMS voltage constraints.pu.csv'

my_FRS_config._set_param('adms_constraint_time', adms_constraint_time)
my_FRS_config._set_param('adms_constraint_file', adms_constraint_file)
my_FRS_config.use_tmm = False
my_FRS_config.use_adms_publisher = True
my_FRS_config.path_to_hotstart_file = str(path_to_here / 'archive' / f"DayAhead_Process_{case}.pkl")
my_FRS_config.file_all_data = str(export_folder / f'{model_name}_{case}_data.pkl')

#Remove TMM from DER list
try:
    static_data['DERs'].pop(mrid_tmm)
except:
    pass

path_to_config_file = export_folder / f'FRS_config_{case}.json'
my_FRS_config.export_config(path_to_config_file)
with open(my_FRS_config.file_all_data, 'wb') as f:
    pickle.dump(static_data, f)
with open(my_FRS_config.file_all_data, 'ab') as f:
    pickle.dump(fcast_data, f)

## PV + BATT

In [ ]:
case = 'PV_BATT'
my_FRS_config.use_tmm = False
my_FRS_config.use_adms_publisher = False
my_FRS_config.path_to_hotstart_file = str(path_to_here / 'archive' / f"DayAhead_Process_{case}.pkl")
my_FRS_config.file_all_data = str(export_folder / f'{model_name}_{case}_data.pkl')

path_to_config_file = export_folder / f'FRS_config_{case}.json'
my_FRS_config.export_config(path_to_config_file)
with open(my_FRS_config.file_all_data, 'wb') as f:
    pickle.dump(static_data, f)
with open(my_FRS_config.file_all_data, 'ab') as f:
    pickle.dump(fcast_data, f)

# PV + BATT + Microgrid

In [ ]:
case = "PV_BATT_MICROGRID"
my_FRS_config.use_mpc_dispatch_controller = True
my_FRS_config.use_microgrid_interface = True
my_FRS_config.path_to_hotstart_file = str(path_to_here / 'archive' / f"DayAhead_Process_{case}.pkl")
my_FRS_config.file_all_data = str(export_folder / f'{model_name}_{case}_data.pkl')

path_to_config_file = export_folder / f'FRS_config_{case}.json'
my_FRS_config.export_config(path_to_config_file)

## Add the Composite Resources
static_data['Composite_Resources'] = {}
# Composite Resource / Microgrid
mRID_CR = 'MICRO_1'
DER_ids = []
Load_ids = []

## Battery 
mRID_BAT = f'{mRID_CR}_BAT'
type = 'BAT'
bus_phase = {'node_634':[0,1,2]}
smax = None #Not really useful for aggregations, though there should be a Pmax.  It is possible that Pmax could just be computed from max capacity across the aggregation.  
qmin = -100 * 1000 #kVAR
qmax = 100 * 1000 #kVAR
eff_c = 0.95
eff_d = 0.95
emax = 375 * 1000 # 80% is 300 kWh
gamma = {'node_634':[50.0, 50.0, 50.0]} #This is actually the max capacity (in KVA) on each phase
E_0 = 0.5*emax

static_data['DERs'][mRID_BAT] = {'type':type, 'bus_phase':bus_phase, 'smax':smax, 'eff_c':eff_c, 'eff_d':eff_d, 'emax':emax, 'gamma':gamma, 'qmin':qmin, 'qmax':qmax, 'E_0':E_0}
# Add to Composite Resource
DER_ids.append(mRID_BAT)

## Load
#Load on bus 634
mRID_LOAD = f'{mRID_CR}_LOAD'
bus = 'node_634'
static_data['Loads'][mRID_LOAD] = {'bus':bus}
# Add to Composite Resource
Load_ids.append(mRID_LOAD)

## Composite Resource in Static Data
static_data['Composite_Resources'][mRID_CR] = {'DER_ids':DER_ids, 'Load_ids':Load_ids}

with open(my_FRS_config.file_all_data, 'wb') as f:
    pickle.dump(static_data, f)

# Add the forecast data
## Load
example_PF = [1, 1, 1]
fcast_data['Loads'][mRID_LOAD] = {}
x_bar = [[50, 50, 50] for t in range(fcast_data['n_timestep'])]
sigma = [[0, 0, 0] for t in range(fcast_data['n_timestep'])]
fcast_data['Loads'][mRID_LOAD]['PL'] = {'xbar':x_bar, 'sigma':sigma}

x_bar_q = [[x_bar[t][idx] * math.tan(math.acos(example_PF[idx])) for idx in [0,1,2]]for t in range(len(x_bar))]
sigma_q = [[sigma[t][idx] * math.tan(math.acos(example_PF[idx])) for idx in [0,1,2]]for t in range(len(sigma))]
fcast_data['Loads'][mRID_LOAD]['QL'] = {'xbar':x_bar_q, 'sigma':sigma_q}

## Battery
fcast = {}
key = 'Emax'
x_bar = [0.9 * 375 *1000]
sigma = [0]
fcast[key] = {'xbar':x_bar, 'sigma':sigma}
key = 'Emin'
x_bar = [0.1 * 375 *1000]
sigma = [0]
fcast[key] = {'xbar':x_bar, 'sigma':sigma}
key = 'Pmax'
x_bar = [150*1000]
sigma = [0]
fcast[key] = {'xbar':x_bar, 'sigma':sigma}
key = 'Pmin'
x_bar = [-150*1000]
sigma = [0]
fcast[key] = {'xbar':x_bar, 'sigma':sigma}

fcast_data['DERs'][mRID_BAT] = fcast

with open(my_FRS_config.file_all_data, 'ab') as f:
    pickle.dump(fcast_data, f)